In [76]:
import cv2
import numpy as np

In [77]:
img = cv2.imread('../input/pos_41.png', cv2.IMREAD_GRAYSCALE)
img = img.astype(float)

In [78]:

h, w = img.shape
center = (w//2, h//2)
theta_1 = -50.0
theta_2 = 50.0
valor_maximo = -1.0
melhor_theta = -11.0
for theta in np.arange(theta_1, theta_2, 1):
    matriz_rotacao = cv2.getRotationMatrix2D(center, theta, 1)
    rotate_img = cv2.warpAffine(img, matriz_rotacao, (w, h), borderMode=cv2.BORDER_REPLICATE)
    perfil = []
    for i in range(h):
        perfil.append(np.sum(rotate_img[i]))
    soma = 0
    for i in range(h-1):
        soma += np.power(perfil[i+1] - perfil[i], 2)
    if (soma > valor_maximo):
        valor_maximo = soma
        melhor_theta = theta
print(melhor_theta)


41.0


In [ ]:

matriz = cv2.getRotationMatrix2D(center, melhor_theta, 1)
rotate_img = cv2.warpAffine(img, matriz, (w, h), borderMode=cv2.BORDER_REPLICATE)
cv2.imwrite('teste.png', rotate_img)

True

: 